In [1]:
pip install -q pyyaml h5py

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.5.0


## Get sample dataset

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
print(test_images.shape)

(10000, 28, 28)


In [6]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0
print(test_images.shape)

(1000, 784)


## define simple Model (Dense, dropout, Dense)

In [7]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model


# Create a basic model instance
model = create_model()
# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## checkpoint : weight만 저장!

In [8]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

Epoch 1/10
32/32 [==============================] - 1s 12ms/step - loss: 2.2918 - sparse_categorical_accuracy: 0.2040 - val_loss: 2.2784 - val_sparse_categorical_accuracy: 0.3850

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 7ms/step - loss: 2.2546 - sparse_categorical_accuracy: 0.3920 - val_loss: 2.2384 - val_sparse_categorical_accuracy: 0.3320

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 2.1944 - sparse_categorical_accuracy: 0.4550 - val_loss: 2.1738 - val_sparse_categorical_accuracy: 0.3960

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 2.1051 - sparse_categorical_accuracy: 0.5300 - val_loss: 2.0826 - val_sparse_categorical_accuracy: 0.4480

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 7ms/step - loss: 1.9865 - sparse_cat

In [12]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [13]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3025 - sparse_categorical_accuracy: 0.1290
Untrained model, accuracy: 12.90%


In [14]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 1.4272 - sparse_categorical_accuracy: 0.6240
Restored model, accuracy: 62.40%


### Honey tip! : filename str format 
-> save filename with epoch

In [15]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [16]:
os.listdir(checkpoint_dir)

['cp-0005.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0045.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0050.ckpt.index']

In [17]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [18]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.5847 - sparse_categorical_accuracy: 0.8240
Restored model, accuracy: 82.40%


## Manually save weights (after learning ends)

In [21]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')
# Create a new model instance
model = create_model()
# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.5847 - sparse_categorical_accuracy: 0.8240
Restored model, accuracy: 82.40%


### Save the entire model

In [22]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 2.2931 - sparse_categorical_accuracy: 0.2380
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 2.2609 - sparse_categorical_accuracy: 0.1910
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 2.2073 - sparse_categorical_accuracy: 0.4140
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 2.1265 - sparse_categorical_accuracy: 0.4320
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 2.0189 - sparse_categorical_accuracy: 0.5310
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [24]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	keras_metadata.pb  saved_model.pb  variables


In [25]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 2.0046 - sparse_categorical_accuracy: 0.5270
Restored model, accuracy: 52.70%
(1000, 10)


## HDF5 format
-weight
- structure of models
- optimizer,loss

In [30]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 2.2925 - sparse_categorical_accuracy: 0.1760
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 2.2579 - sparse_categorical_accuracy: 0.2340
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 2.2042 - sparse_categorical_accuracy: 0.3610
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 2.1260 - sparse_categorical_accuracy: 0.4810
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 2.0187 - sparse_categorical_accuracy: 0.4980


In [31]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [32]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 2.0039 - sparse_categorical_accuracy: 0.4950
Restored model, accuracy: 49.50%
